# Venus8 GPS

Comercially availible GPS receivers are problematic for rockets. For technical reasons filters and satellite tracking loops in off the shelf GPS chips are focused on surface uses. Even for aviation grade GPS (much, more expensive!) they do not expect you to accelerate straight up at 10 g's.

Aside from the technical challenge of keeping lock under high g-load, there are legal restrictions in place for GPS as well. Usually refered to "COCOM limits", all purchased GPS units must stop giving valid data above certain speeds and altitudes.

This makes things very difficult on a rocket. Often you run into COCOM limits and acceleration errors at the same time (high accleration and speed).

## Reported Fix Information

The Venus8 chip will report how good a fix it has as it goes. We can plot both the number or locked satellites and the "Fix Mode" which will tell us "no lock", "2D fix", or various grades of full position fix modes.

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, average, arange, absolute
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

t_0 = 117853569585227

columns = loadtxt("../fc-data/V8A8.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
fix_mode = columns[2]
num_sv = columns[3]
latitude = columns[6]
longitude = columns[7]
altmsl = columns[9]
GDOP = columns[10]
PDOP = columns[1]
HDOP = columns[13]
VDOP = columns[14]
TDOP = columns[15]
vel_x = columns[18]
vel_y = columns[19]
vel_z = columns[20]

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)


columns = loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True)

acctime = columns[1]
acc_x = columns[6]

acctime = subtract(acctime, t_0)
acctime = divide(acctime, 1e9)

# lightly lowpass
from scipy.signal import butter, lfilter, freqz

# Filter requirements.
order = 6
fs = 819.2       # sample rate, Hz
cutoff = 20   # desired cutoff frequency of the filter, Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq

# Get the filter coefficients so we can check its frequency response.
b, a = butter(order, normal_cutoff, btype='low', analog=False)
acc_x_filter = lfilter(b, a, acc_x)


fig, ax1 = plt.subplots(figsize=(16,9))
plt.title(r"Venus 8 GPS Reciever Reported Fix ")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"#")
ax1.plot(timestamp, num_sv, alpha=0.75, label="Number of Sats locked")
ax1.plot(timestamp, fix_mode, lw=3, alpha=0.75, label="Reported Fix Mode (2=3D Fix, 3=3D Fix + DGPS)")


ax2 = ax1.twinx()
ax2.set_ylabel(r"Acceleration [m/s${}^2$]")
ax2.plot(acctime[200:-200], acc_x_filter[200:-200], 'k-', alpha=0.18, label="IMU Acceleration")
ax2.grid(b=False)
ax2.set_ylim([-50,150])


ax1.set_xlim([-5,42])
ax2.set_xlim([-5,42])
ax1.set_ylim([0,12])
ax1.legend(loc=2)
ax2.legend(loc=1)
plt.show()

At least initially, it would look like the Venus tracked the entire flight! Though, well see this is in fact not true.

In GPS we can also get reports of the "Dilution of Precision" (DOP) over time. This is related to how good a constellation you have at any moment and some errors in the psuodoranges to the satellites. Lower numbers are better.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
plt.title(r"Venus 8 GPS Reciever DOP")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Dilution of Precision")
ax1.plot(timestamp, GDOP, alpha=0.75, label="GDOP")
ax1.plot(timestamp, VDOP, alpha=0.75, label="VDOP")
ax1.plot(timestamp, HDOP, alpha=0.75, label="HDOP")
#ax1.set_ylim([0,12])


ax2 = ax1.twinx()
ax2.set_ylabel(r"Acceleration [m/s${}^2$]")
ax2.plot(acctime[200:-200], acc_x_filter[200:-200], 'k-', alpha=0.18, label="IMU Acceleration")
ax2.grid(b=False)
ax2.set_ylim([-50,150])

ax1.set_xlim([-5,42])
ax2.set_xlim([-5,42])
ax1.legend(loc=2)
ax2.legend(loc=1)
plt.show()

Again the Venus misleadingly reports very good results.

## Altitude

If the fix is good, we should be able to see the rocket flight in high precision. Let's look at the reported altitdue from the Venus8 GPS chip.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
plt.title(r"Venus 8 GPS Reciever Altitude")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Altitude AGL [km]")
ax1.plot(timestamp, divide(subtract(altmsl, 1390), 1000.0), alpha=0.75)
ax1.set_xlim([-5,42])
#ax1.set_ylim([0,12])
plt.show()

Oops. The venus does not report altitude for nearly the **entire** flight. There is also a mysterious instantaneous vertical bump t+31 seconds.

Though we can see the exact altitude of the GPS antenna just before launch above sea level:

In [ ]:
print "MSL Altitude pre-launch:  %0.3f m" % average(altmsl[:30])

## Position

We also get lat/lon pairs for the flight.

In [ ]:
from mpl_toolkits.basemap import Basemap
import json

with open('map.json', 'r') as geojson:
    geo = json.loads(geojson.read())

    roads = []
    for coll in geo['features']:
        if coll[u'properties'][u'description'] == "road":
            lat = [c[1] for c in coll[u'geometry'][u'coordinates']]
            lon = [c[0] for c in coll[u'geometry'][u'coordinates']]
            roads.append((lon, lat))

ax = plt.figure(figsize=(16,16))
plt.title("Launch Site")

m = Basemap(projection='aea',
            lat_0=43.798,lon_0=-120.6517534,
            width=1800,height=1100,resolution ='c')

m.drawparallels(arange(  43.4,  44.0,0.002),labels=[1,1,0,0], color='#cccccc', dashes=(3,7))
m.drawmeridians(arange(-120.8,-120.4,0.002),labels=[0,0,0,1], color='#cccccc', dashes=(3,7))

                        
mx, my = m(-120.64904987812042, 43.798809500361756)
plt.text(mx, my, "Flight Line", color="#444444", rotation=-13, ha='center', va='center')

for road in roads:
    rx, ry = m(road[0], road[1])
    m.plot(rx,ry, color="#cccccc", alpha=1, lw=3.0)
    
x, y = m(longitude, latitude)
m.plot(x,y,'b-', alpha=0.1)
m.plot(x,y,'r.')


mx, my = m(longitude[0], latitude[0])
m.plot(mx, my, '+', color="#999999", markersize=20, markeredgewidth=1.0)
plt.text(mx-5, my-5, "Launch Tower", color="#444444", rotation=45, ha='right', va='top')

    
plt.show()

The flight trace is in red, and gaps (thin blue line connects where the trace should cover) are where the Venus again reports the same position throughout the flight until it suddently snaps to the correct position at apogee.


## Velocity

GPS can actually determine velocity separately from postition by comparing the doppler shifts from each satellite. Here is the total velocity over time as reported by Venus

In [ ]:
from math import sqrt

velocity = []
for i, t in enumerate(timestamp):
    v = sqrt((vel_x[i]*vel_x[i]) + (vel_y[i]*vel_y[i]) + (vel_z[i]*vel_z[i]))
    velocity.append(v)
    
columns = loadtxt("uncalibrated_integrated_velocity.csv", delimiter=',', unpack=True)
alt_time = columns[0]
imuvel   = columns[1]

fig, ax1 = plt.subplots(figsize=(16,9))
plt.title(r"Venus 8 GPS Reciever Altitude")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Altitude AGL [km]")
ax1.plot(alt_time, absolute(imuvel), 'k-', alpha=0.2, label="IMU Integrated Vertical Velocity")
ax1.plot(timestamp, velocity, alpha=0.75, label="Venus8 GPS Velocity Magnitude")
ax1.set_xlim([-5,42])
#ax1.set_ylim([0,12])
ax1.legend()
plt.show()

Another case of bogus data. Clearly the Venus8 lost track of everything during the launch and only reconnected after the flight was over and it was starting to open parachutes.